In [269]:
import pandas as pd
import json

pd.set_option('display.max_columns', None)
all_df = pd.read_csv('./data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1", keep_default_na=False)
lat_lon_code = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv', keep_default_na=False)
continent = pd.read_csv('./data/UNSD — Methodology.csv', sep=';', encoding='utf-8', keep_default_na=False)

lat_lon_code = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv')
lat_lon_code = lat_lon_code.sort_values(by=['Country']).drop_duplicates(subset=['Alpha-2 code'])
lat_lon_code.loc[:, "Numeric code"] = lat_lon_code["Numeric code"].str.strip(' "').astype(int)
lat_lon_code.loc[:, "Alpha-2 code"] = lat_lon_code["Alpha-2 code"].str.strip(' "')
lat_lon_code.loc[:, "Latitude (average)"] = lat_lon_code["Latitude (average)"].str.strip(' "').astype(float)
lat_lon_code.loc[:, "Longitude (average)"] = lat_lon_code["Longitude (average)"].str.strip(' "').astype(float)
area_code_to_iso_dict = lat_lon_code[["Numeric code", "Alpha-2 code"]].set_index("Numeric code")['Alpha-2 code'].to_dict()

all_df.loc[:, 'Area Code (M49)'] = all_df['Area Code (M49)'].str.strip(" '").astype(int)
all_df.loc[:, 'ISO-A2'] = all_df['Area Code (M49)'].replace(area_code_to_iso_dict)
all_df = all_df[all_df['ISO-A2'].isin(lat_lon_code['Alpha-2 code'])]

def make_year_cols(start, end):
    return [f'Y{y}' for y in range(start, end)]

In [270]:
all_df = all_df.merge(
    lat_lon_code[['Latitude (average)', 'Longitude (average)', 'Country', 'Alpha-2 code']],
    how='left',
    left_on='ISO-A2',
    right_on='Alpha-2 code',
)

all_df = all_df.melt(
    id_vars=['Country', 'ISO-A2', "Longitude (average)", "Latitude (average)", 'Item', 'Item Code', 'Element', 'Unit'], 
    value_vars=make_year_cols(2000, 2021), 
    value_name='Quantity', 
    var_name='Year'
)

all_df['Year'] = all_df['Year'].str.strip('Y').astype(int)
all_df = all_df.rename(columns={"Country": "Area"})
all_df.loc[:, 'Area'] = all_df['Area'].apply(lambda x: x if not x.endswith('of') and not x.endswith('of the') else ' '.join(x.split(', ')[::-1]))

In [ ]:
# population = population.melt(
#     id_vars=['Country Name'],
#     value_vars=[str(y) for y in range(2000, 2021)],
#     value_name='Population',
#     var_name='Year',
# )

# population['Year'] = population['Year'].astype(int)
# all_df = pd.merge(
#     all_df, population,
#     how='left',
#     left_on=['Area', 'Year'],
#     right_on=['Country Name', 'Year']
# )

In [271]:
all_df = all_df.merge(
    continent[['Region Name', 'ISO-alpha2 Code']],
    how='left',
    left_on='ISO-A2',
    right_on='ISO-alpha2 Code'
).rename(columns={'Region Name': 'Continent'}).drop(columns=['ISO-alpha2 Code'])

# check country name

In [24]:
with open('./data/ne_110m.json', 'r') as f:
    text = f.read()

In [ ]:
for country_name in all_df['Area'].unique():
    if text.find('"NAME":"{}"'.format(country_name)) == -1:
        print(country_name)


In [253]:
all_df[(all_df['Area'] == 'Taiwan') & (all_df['Item'].str.endswith('Total'))]

,Area,ISO-A2,Longitude (average),Latitude (average),Item,Item Code,Element,Unit,Year,Quantity,Continent
10705,Taiwan,TW,121.0,23.5,"Cereals, Total",1717,Area harvested,ha,2000,375703.0,NaN
10706,Taiwan,TW,121.0,23.5,"Cereals, Total",1717,Yield,hg/ha,2000,56224.0,NaN
10707,Taiwan,TW,121.0,23.5,"Cereals, Total",1717,Production,tonnes,2000,2112369.0,NaN
10708,Taiwan,TW,121.0,23.5,"Citrus Fruit, Total",1804,Area harvested,ha,2000,32481.0,NaN
10709,Taiwan,TW,121.0,23.5,"Citrus Fruit, Total",1804,Yield,hg/ha,2000,139378.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1144649,Taiwan,TW,121.0,23.5,"Pulses, Total",1726,Yield,hg/ha,2020,20324.0,NaN
1144650,Taiwan,TW,121.0,23.5,"Pulses, Total",1726,Production,tonnes,2020,12784.0,NaN
1144651,Taiwan,TW,121.0,23.5,"Roots and Tubers, Total",1720,Area harvested,ha,2020,15191.0,NaN
1144652,Taiwan,TW,121.0,23.5,"Roots and Tubers, Total",1720,Yield,hg/ha,2020,227325.0,NaN


# wrangle file

In [250]:
df = all_df[
    (all_df['Item'].str.endswith('Total')) &\
    (all_df['Element'].isin(['Production']))
].dropna(
  axis=0,
  how='any',
)

df.to_csv('data/processed/production_major_categories_per_country_2000-2020.csv')

In [251]:
df = all_df[
    (all_df['Item'].str.endswith('Total')) &\
    (all_df['Element'].isin(['Yield']))
].dropna(
  axis=0,
  how='any',
)

df.to_csv('data/processed/yield_major_categories_per_country_2000-2020.csv')

In [208]:
df = all_df[
    (~all_df['Item'].str.endswith('Total')) &\
    (all_df["Element"] == "Production")
].dropna(
  axis=0,
  how='any',
).groupby(["Item", "Year", "Unit"])[["Quantity"]]\
 .sum()\
 .reset_index()

df.to_csv('./data/processed/production_all_category_2000_2020.csv')

In [ ]:
1) CROPS PRIMARY:
['Cereals',' Citrus Fruit',' Fibre Crops', 'Fruit', 'Oil Crops', 'Oil Crops and Cakes in Oil Equivalent',' Pulses',' Roots and Tubers',' Sugar Crops',' Treenuts and Vegetables']
2) CROPS PROCESSED:
Beer of barley; Cotton lint; Cottonseed; Margarine, short; Molasses; Oil, coconut (copra); Oil, cottonseed; Oil, groundnut; Oil, linseed; Oil, maize; Oil, olive, virgin; Oil, palm; Oil, palm kernel; Oil, rapeseed; Oil, safflower; Oil, sesame; Oil, soybean; Oil, sunflower; Palm kernels; Sugar Raw Centrifugal; Wine.
3) LIVE ANIMALS: Animals live n.e.s.; Asses; Beehives; Buffaloes; Camelids, other; Camels; Cattle; Chickens; Ducks; Geese and guinea fowls; Goats; Horses; Mules; Pigeons, other birds; Pigs; Rabbits and hares; Rodents, other; Sheep; Turkeys.
4) LIVESTOCK PRIMARY: Beeswax; Eggs (various types); Hides buffalo, fresh; Hides, cattle, fresh; Honey, natural; Meat (ass, bird nes, buffalo, camel, cattle, chicken, duck, game, goat, goose and guinea fowl, horse, mule, Meat nes, meat other camelids, Meat other rodents, pig, rabbit, sheep, turkey); Milk (buffalo, camel, cow, goat, sheep); Offals, nes; Silk-worm cocoons, reelable; Skins (goat, sheep); Snails, not sea; Wool, greasy.
5) LIVESTOCK PROCESSED: Butter (of milk from sheep, goat, buffalo, cow); Cheese (of milk from goat, buffalo, sheep, cow milk); Cheese of skimmed cow milk; Cream fresh; Ghee (cow and buffalo milk); Lard; Milk (dry buttermilk, skimmed condensed, skimmed cow, skimmed dried, skimmed evaporated, whole condensed, whole dried, whole evaporated); Silk raw; Tallow; Whey (condensed and dry); Yoghurt

In [57]:
import json
with open('./data/ne_110m_admin_0_countries.topojson', 'r', encoding="ISO-8859-1") as f:
    x = json.load(f)

In [58]:
f = [geom['properties']['NAME'] for geom in x['objects']['ne_110m_admin_0_countries']['geometries']]

In [59]:
set(f).difference(set(all_df["Area"]))

{'Antarctica',
 "CÃ´te d'Ivoire",
 'Falkland Is.',
 'Fr. S. Antarctic Lands',
 'Greenland',
 'Kosovo',
 'Macedonia',
 'N. Cyprus',
 'S. Sudan',
 'Somaliland',
 'W. Sahara',
 'eSwatini'}

In [45]:
set(f)

ValueError: I/O operation on closed file.

In [69]:
all_df[all_df['Item'].str.endswith('Primary')]["Item"].unique()

array(['Eggs Primary', 'Fruit Primary', 'Sugar Crops Primary',
       'Vegetables Primary'], dtype=object)